In [ ]:
!pip install tensorflow-gpu==1.15
!pip install tf-nightly-gpu

In [ ]:
import tensorflow as tf
import os

In [ ]:
import cv2 as cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pathlib

interpreter = tf.lite.Interpreter(model_path="/content/converted_model.tflite")

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.allocate_tensors()

def draw_rect(image, box):
    y_min = int(300 * box[0])
    x_min = int(300 * box[1])
    y_max = int(300 * box[2])
    x_max = int(300 * box[3])
    
    cv2.rectangle(image, (x_min, y_min),(x_max, y_max), (255, 0, 0), 2)

for file in pathlib.Path('/content/drive/My Drive/dataset').iterdir():

    if file.suffix != '.jpg' and file.suffix != '.png':
        continue

    img = cv2.imread(r"{}".format(file.resolve()))
    new_img = cv2.resize(img, (300, 300))
    interpreter.set_tensor(input_details[0]['index'], [new_img])

    interpreter.invoke()

    rects = interpreter.get_tensor(output_details[0]['index'])

    scores = interpreter.get_tensor(output_details[2]['index'])

#    print(rects.shape)
  #    print(scores.shape)
    max_v = 0
    ind = 0
    for i in range(len(rects[0])):
      if max_v < np.max(rects[0][i]):
        max_v = np.max(rects[0][i])
        ind = i
    # print(f"max value: {max_v} at line {ind}")
    # print(scores[0][ind])
    draw_rect(new_img,scores[0][ind])
    # for index, score in enumerate(scores[0]):
    #     if score.all() > 0.8:
    #       draw_rect(new_img,rects[0][index])
    cv2_imshow(new_img)
    cv2.waitKey(0)